# VGG

## CIFAR10介绍
cifar 10 这个数据集一共有 50000 张训练集，10000 张测试集，两个数据集里面的图片都是 png 彩色图片，图片大小是 32 x 32 x 3，一共是 10 分类问题，分别为飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船和卡车。这个数据集是对网络性能测试一个非常重要的指标，可以说如果一个网络在这个数据集上超过另外一个网络，那么这个网络性能上一定要比另外一个网络好，目前这个数据集最好的结果是 95% 左右的测试集准确率。

![](https://ws1.sinaimg.cn/large/006tNc79ly1fmpjxxq7wcj30db0ae7ag.jpg)
cifar 10 已经被 pytorch 内置了，使用非常方便，只需要调用 `torchvision.datasets.CIFAR10` 就可以了

## VGGNet介绍
vggNet 是第一个真正意义上的深层网络结构，其是 ImageNet2014年的冠军，得益于 python 的函数和循环，我们能够非常方便地构建重复结构的深层网络。

vgg 的网络结构非常简单，就是不断地堆叠卷积层和池化层，下面是一个简单的图示

![](https://ws4.sinaimg.cn/large/006tNc79ly1fmpk5smtidj307n0dx3yv.jpg)

vgg 几乎全部使用 3 x 3 的卷积核以及 2 x 2 的池化层，使用小的卷积核进行多层的堆叠和一个大的卷积核的感受野是相同的，同时小的卷积核还能减少参数，同时可以有更深的结构。

vgg 的一个关键就是使用很多层 3 x 3 的卷积然后再使用一个最大池化层，这个模块被使用了很多次，下面我们照着这个结构来写一写

## VGG代码实现

In [4]:
import sys
sys.path.append('..')

import numpy as np
import torch 
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [6]:
def vgg_block(num_convs,in_channels,out_channels):
    net=[nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1),nn.ReLU(True)] #定义第一层
    for i in range(num_convs-1):
        net.append(nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1))
        net.append(nn.ReLU(True))
    net.append(nn.MaxPool2d(2,2))#最后最大池化,长度宽度要减半
    net_real=nn.Sequential(*net) # nn.Sequential(*list)就是解包操作.因为nn.Sequential()不能接受list,只能将list解包再送入nn.Sequential()中.
    return  net_real

In [10]:
def vgg_stack(num_convs,channels):
    net=[]
    for num,channel in zip(num_convs,channels):
        input_channel=channel[0]
        output_channel=channel[1]
        net.append(vgg_block(num,input_channel,output_channel))
        net_real=nn.Sequential(*net)
    return net_real

In [18]:
vgg_net = vgg_stack((1, 1, 2, 2, 2), ((3, 64), (64, 128), (128, 256), (256, 512), (512, 512)))
# print(vgg_net)

In [19]:
test_x = Variable(torch.zeros(1, 3, 256, 256))
test_y = vgg_net(test_x)
print(test_y.shape)

torch.Size([1, 512, 8, 8])


In [31]:
class vgg(nn.Module):
    def __init__(self):
        super(vgg,self).__init__()
        self.feature=vgg_net
        self.fc=nn.Sequential(nn.Linear(512,100),
                             nn.ReLU(True),
                             nn.Linear(100,10))
    def forward(self,x):
        x=self.feature(x)
        x=x.view(x.shape[0],-1)
        x=self.fc(x)
        return x

In [24]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    x = x.transpose((2, 0, 1)) # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = torch.from_numpy(x)
    return x
     
train_set = CIFAR10('./data', train=True, transform=data_tf,download=True)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf,download=True)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [34]:
def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().item()
    return num_correct / total
def train(net, train_data, test_data, epoch, optimizer, criterion):
    if torch.cuda.is_available():
        net=net.cuda()
    for e in range(epoch):
        train_loss=0
        train_acc=0
        net=net.train()
        for im,label in train_data:
            if torch.cuda.is_available():
                im=Variable(im.cuda())
                label=Variable(label.cuda())      
            else:
                im=Variable(im)
                label=Variable(label)
            #forward
            output=net.forward(im)#此处有问题....
            loss=criterion(output,label)
            #backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
            train_acc+=get_acc(output,label)
        print(train_loss/len(train_data))

net = vgg()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()
train(net, train_data, test_data, 20, optimizer, criterion)

2.3031342105792305
2.3030203254631414
2.302961255888195
2.3026441344824593
2.141859237037961
1.7751500028783402
1.53880198517114
1.329759924384334
1.1291363163829764
0.9430304761128048
0.7874331894280661
0.6547647973384394
0.5450474916173674
0.4433809178678886
0.3510578895640343
0.2852087345098138
0.22053750786844575
0.17837492057391444
0.14442929543573838
0.11171062031517857
